In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("alltogether2.4v.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]


In [4]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(label_encoder.classes_)

# Normalize features
X_train_normalized = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test_normalized = (X_test - X_train.min()) / (X_train.max() - X_train.min())



# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation="softmax")
])

['ALU' 'HDPE' 'LDPE' 'PP']


In [5]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_normalized, y_train, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])
# history = model.fit(X_train_normalized, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)




Epoch 1/200
2/2 [==============================] - 1s 176ms/step - loss: 1.3850 - accuracy: 0.3193 - val_loss: 1.3255 - val_accuracy: 0.3333
Epoch 2/200
2/2 [==============================] - 0s 26ms/step - loss: 1.3472 - accuracy: 0.4034 - val_loss: 1.2855 - val_accuracy: 0.6333
Epoch 3/200
2/2 [==============================] - 0s 26ms/step - loss: 1.3157 - accuracy: 0.5210 - val_loss: 1.2551 - val_accuracy: 0.6333
Epoch 4/200
2/2 [==============================] - 0s 33ms/step - loss: 1.2927 - accuracy: 0.5210 - val_loss: 1.2305 - val_accuracy: 0.6333
Epoch 5/200
2/2 [==============================] - 0s 31ms/step - loss: 1.2714 - accuracy: 0.5462 - val_loss: 1.2083 - val_accuracy: 0.6667
Epoch 6/200
2/2 [==============================] - 0s 29ms/step - loss: 1.2502 - accuracy: 0.5966 - val_loss: 1.1845 - val_accuracy: 0.6667
Epoch 7/200
2/2 [==============================] - 0s 36ms/step - loss: 1.2265 - accuracy: 0.6050 - val_loss: 1.1581 - val_accuracy: 0.7000
Epoch 8/200
2/2 [==

In [6]:
# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)



# Convert predictions to percentage likelihood
percentage_likelihood = predictions.max(axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(15):
    if y_test[i] == predicted_labels[i]:
        accuracy = "Correct"
    else:
        accuracy = "False"
    print(f"Sample {i+1}: {accuracy} Predicted {predicted_plastic_types[i]} with {percentage_likelihood[i]:.2f}% likelihood")


# Save the model
model.save("plastic_classifier_model.h5")

total_params = model.count_params()
total_size_bytes = total_params * 4  # Each parameter is usually a 32-bit float
total_size_mb = total_size_bytes / (1024)  # Convert bytes to megabytes

print(f"Total size of the model: {total_size_mb:.2f} KB")


2/2 [==============================] - 0s 2ms/step
Test Accuracy: 0.9210526347160339
Sample 1: Correct Predicted ALU with 99.43% likelihood
Sample 2: Correct Predicted LDPE with 75.84% likelihood
Sample 3: Correct Predicted LDPE with 70.82% likelihood
Sample 4: Correct Predicted PP with 87.13% likelihood
Sample 5: Correct Predicted ALU with 99.96% likelihood
Sample 6: Correct Predicted HDPE with 99.61% likelihood
Sample 7: Correct Predicted LDPE with 70.79% likelihood
Sample 8: Correct Predicted ALU with 99.94% likelihood
Sample 9: Correct Predicted PP with 53.74% likelihood
Sample 10: Correct Predicted ALU with 99.88% likelihood
Sample 11: Correct Predicted ALU with 99.96% likelihood
Sample 12: Correct Predicted HDPE with 74.76% likelihood
Sample 13: Correct Predicted ALU with 99.87% likelihood
Sample 14: Correct Predicted HDPE with 93.12% likelihood
Sample 15: Correct Predicted LDPE with 92.33% likelihood
Total size of the model: 74.52 KB


c:\Users\kylej\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -4:]
top_labels = top_predictions[:, ::-1]

print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(5):
    num_predictions = min(4, len(np.unique(top_labels[i])))
    print(f"Sample {i+1}:")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][j]:.2f}% likelihood")

# Save the model
model.save("plastic_classifier_model.h5")

2/2 [==============================] - 0s 2ms/step
[[9.94296134e-01 7.20957702e-04 4.98289149e-03 4.93897839e-11]
 [1.50635757e-03 3.12804896e-03 7.58433461e-01 2.36932069e-01]
 [2.40687965e-04 4.28390503e-03 7.08185792e-01 2.87289649e-01]
 [1.36413117e-04 2.80180102e-04 1.28329217e-01 8.71254086e-01]
 [9.99583423e-01 2.76171363e-07 4.16291732e-04 1.74762006e-12]
 [6.08490018e-06 9.96096194e-01 6.68732275e-04 3.22898314e-03]
 [6.31820178e-04 5.05661294e-02 7.07902074e-01 2.40899950e-01]
 [9.99417782e-01 2.43909255e-07 5.82004141e-04 2.75361244e-12]
 [2.70551327e-03 2.03924114e-03 4.57847416e-01 5.37407875e-01]
 [9.98834789e-01 1.82792373e-05 1.14684843e-03 2.56719403e-11]
 [9.99643564e-01 4.94430139e-07 3.55971191e-04 1.05906668e-12]
 [2.53414764e-04 7.47634292e-01 3.50119621e-02 2.17100292e-01]
 [9.98747230e-01 1.85280551e-05 1.23422651e-03 3.94628982e-11]
 [2.69023760e-04 9.31222737e-01 3.67293954e-02 3.17787826e-02]
 [5.47996024e-03 5.01889642e-03 9.23266053e-01 6.62350357e-02]
 [7.

c:\Users\kylej\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Get manual input from the user
manual_input = np.array([[400255,937687,1955595,3318033,3826709,2380639,1264331,783625,939273,1956581,3319213,5576989,2381183,1264955,782841]])  # Replace with your input values

# Normalize the manual input if needed
normalized_manual_input = ((manual_input - 0) / (8388608 - 0))

# Make predictions on manual input
predictions = model.predict(normalized_manual_input)
print(predictions)

# Get the top prediction
top_prediction = np.argmax(predictions)
print(top_prediction)

top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
print(top_plastic_type)
likelihood = predictions[0][top_prediction] * 100

# Display the prediction for manual input
print(f"Manual Input Prediction:")
print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

1/1 [==============================] - 0s 47ms/step
[[0.41227806 0.48141742 0.03893103 0.06737347]]
1
HDPE
Manual Input Prediction:
  Prediction: HDPE with 48.14% likelihood


In [9]:
# Make predictions from manual
manual_input = np.array([[1047055,2403409,3929955,4807085,6261658,5048526,2421014,1641268,1309392,2610106,3863733,5212999,5402417,3628861,2006927
]])

# Calculate the column-wise min and max values from your original dataset
min_values = [74296, 1151721, 2379300, 2748695, 3681840, 1997402, 1742243, 942818, 392145, 795677, 1406647, 2482140, 2630495, 1748806, 1249297]
max_values = [1773075, 3755206, 7787954, 8401566, 8405071, 8401566, 7232265, 8185313, 2080604, 4455589, 8402979, 8405071, 8405071, 8403086, 6084492]

# Convert the lists to numpy arrays
min_values = np.array(min_values)
max_values = np.array(max_values)

# Normalize the manual input using the calculated min and max values
normalized_manual_input = (manual_input - min_values) / (max_values - min_values)
# normalized_manual_input = np.array([[0.85611,0.63202,0.53239,0.41068,0.59174,0.39869,0.44654,0.42188,0.8826042,0.67470967,0.34701898,0.42024683,0.34977269,0.16862726,0.1471771]])

print(normalized_manual_input)

predictions = model.predict(normalized_manual_input)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

    
## Calculate the percentage likelihoods
percentage_likelihoods = predictions * 100

# Get the indices of the top 4 predicted classes based on percentage likelihoods
top_indices = np.argsort(percentage_likelihoods[0])[::-1][:4]

# Get the labels for the top predicted classes
top_labels = label_encoder.inverse_transform(top_indices)

# Get the predicted label
predicted_label = top_labels[0]

# Get the true label from the original data (replace with your true label)
true_label = 'ALU'  # Replace with the true label

# Calculate the accuracy
if predicted_label == true_label:
    accuracy = "Correct"
else:
    accuracy = "False"

# Print the prediction, label, and accuracy
print(f"Predicted label: {predicted_label}")
print(f"True label: {true_label}")
print(f"Accuracy: {accuracy}")

# Print the percentages for each type
for label, likelihood in zip(top_labels, percentage_likelihoods[0][top_indices]):
    print(f"{label}: {likelihood:.2f}% likelihood")

# ... (rest of the code)

[[0.57262245 0.48077404 0.28669887 0.36413178 0.54619772 0.47642815
  0.12363721 0.09643776 0.54324505 0.49575755 0.35119631 0.46106548
  0.48002174 0.2825332  0.15669068]]
1/1 [==============================] - 0s 19ms/step
Predicted label: HDPE
True label: ALU
Accuracy: False
HDPE: 98.03% likelihood
LDPE: 1.25% likelihood
ALU: 0.71% likelihood
PP: 0.02% likelihood
